In [0]:
#Packages to Import
import pandas as pd
from google.colab import files
import numpy as np
from sklearn.cross_validation import train_test_split
from scipy import optimize as op
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import io
import itertools
import numpy as np

In [0]:
!pip install lmfit
import lmfit

In [0]:
#Uploading File
uploaded = files.upload()

In [0]:
sample_df = pd.read_csv(io.StringIO(uploaded['owls.csv'].decode('utf-8')),header=None)
print(sample_df)

In [0]:
#Adding Attributes to the dataset
sample_df.columns=["Body-length","Wing-length","Body-width","Wing-width","Type"]
sample_df

In [0]:
#optional-Scatter plot for Boady length and wing length
OwlLength = sb.FacetGrid(sample_df, hue="Type", size=6).map(plt.scatter, "Body-length", "Wing-length")
plt.legend(loc='upper left');

In [0]:
#optional-Scatter plot for Boady width and wing width
OwlLength = sb.FacetGrid(sample_df, hue="Type", size=6).map(plt.scatter, "Body-width", "Wing-width")
plt.legend(loc='upper left');

In [0]:
size = sample_df.shape[0] #size of sample
features = 4  #number of features
predictors = 3  #number of types(predictors)


In [0]:
z= np.ones((135,1))
sample_df=pd.DataFrame(z).join(sample_df)
sample_df

In [0]:
X = np.ones((size,features + 1))
y = np.array((size,1))
X[:,1] = sample_df['Body-length'].values 
X[:,2] = sample_df['Wing-length'].values
X[:,3] = sample_df['Body-width'].values
X[:,4] = sample_df['Wing-width'].values
print(X)


In [0]:
#Splitting Test and Training data
df=sample_df.sample(frac=1).reset_index(drop=True)
X=np.array(df.iloc[:,:5])
y=np.array(df.iloc[:,5])
X_test,X_train=X[:45,:],X[45:,:]
y_test,y_train=y[:45],y[45:]
y = ['BarnOwl', 'LongEaredOwl', 'SnowyOwl']
y

In [0]:
#Normalization of Data
for k in range(features):
    X[:, k] = (X[:, k] - X[:,k].mean())
  
print(X)


In [0]:
#Sigmoid fuction
def sigmoid(z):
    f= 1.0 / (1 + np.exp(-z))
    return f


#Regularized cost function
def regCostFunction(params, beta, X, y,lr = 0.1):
    m = len(y)
    dot_X=[]
    dot_X=np.dot(X,beta)
    h = sigmoid(dot_X)
    reg = (lr/(2 * m)) * np.sum(beta**2)

    return (1 / m) * (-y.T.dot(np.log(h)) - (1 - y).T.dot(np.log(1 - h))) + reg

#Regularized gradient function
def regGradient(beta, X, y, lr = 0.1):
    m, n = X.shape
    beta = beta.reshape((n, 1))
    y = y.reshape((m, 1))
    h = sigmoid(X.dot(beta))
    reg = lr * beta /m

    return ((1 / m) * X.T.dot(h - y)) + reg

#Optimal beta 
def logisticRegression(X, y, beta):
  param=lmfit.Parameters()
  #result = lmfit.minimize(fun = regCostFunction, x0 = beta, args = (X, y), method = 'TNC', jac = regGradient)
  minner = lmfit.Minimizer(regCostFunction, param, fcn_args=(beta, X, y))
  result = minner.minimize(method = 'tnc')
  print(result.message)
  print(result.success)
  print(result.status)
  print(result.nfev)
  return result.x

In [0]:
as_beta = np.zeros((k, features + 1))

#One vs all
i = 0
for j in y:
    #set the labels in 0 and 1
    temp_y = np.array(y_train == j, dtype = int)
    optimalBeta= logisticRegression(X_train, temp_y, np.zeros((features + 1,1)))
    as_beta[i] = optimalBeta
    i += 1

In [0]:
#Accuracy Calculation from Scratch

P = sigmoid(X_train.dot(as_beta.T)) 
p = [y[np.argmax(P[i, :])] for i in range(X_train.shape[0])]
accuracy=np.mean(p==y_train)
#res=accuracy*100
print("Model Accuracy from Scratch ", accuracy * 100 , '%')

In [0]:
print("Model Accuracy using scikit-learn ", accuracy_score(y_train, p) * 100 , '%')